In [8]:
# Used Packages
# Used Packages
import pandas as pd         # For data manipulation
%pip install  -q numpy
%pip install -q tensorflow

# Ensure numpy is installed before importing
import numpy as np          # For numerical computations
import matplotlib.pyplot as plt  # For data visualization
import seaborn as sns       # For enhanced visualizations
from nltk.corpus import stopwords  # For stopword removal
from nltk.tokenize import word_tokenize  # For tokenization
import nltk                 # For text processing tools
from sklearn.model_selection import train_test_split  # For splitting datasets
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # For evaluation
from gensim.models import Word2Vec  # For word embeddings
from tensorflow.keras.models import Sequential  # For building neural networks
from tensorflow.keras.layers import Dense, Embedding, LSTM  # For model layers
from tensorflow.keras.preprocessing.text import Tokenizer  # For text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences  # For sequence padding

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ImportError: cannot import name 'triu' from 'scipy.linalg' (/opt/anaconda3/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [ ]:
# Load the dataset
dataset_path = 'data/sentiment140.csv'  # Adjust the path as necessary
columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']  # Column names for the dataset
df = pd.read_csv(dataset_path, encoding='latin1', names=columns)

In [ ]:
df.head(-5)

In [ ]:
# Keep only relevant columns
df = df[['sentiment', 'text']]

# Map sentiment values: 0 = negative, 2 = neutral, 4 = positive
df['sentiment'] = df['sentiment'].map({0: 'negative', 2: 'neutral', 4: 'positive'})

# Display the updated dataset
print("\nCleaned Dataset Preview:")
print(df.head())
